In [33]:
import torch
import os
import torch.nn as nn

class LRN(nn.Module):
    def __init__(self,local_size=1,alpha=1.0,beta=0.75,ACROSS_CHANNELS=True):
        super(LRN,self).__init__()
        self.ACROSS_CHANNELS=ACROSS_CHANNELS
        self.alpha=alpha
        self.beta=beta
        if ACROSS_CHANNELS:
            self.average=nn.AvgPool3d(kernel_size=(local_size,1,1),
                                     stride=1,padding=(int((local_size-1.0)/2),0,0))
        else:
            self.average=nn.AvgPool2d(kernel_size=local_size,stride=1,padding=(int((local_size-1.0)/2),0,0))
            self.alpha=alpha
            self.beta=beta
            
    def forward(self,x):
        if self.ACROSS_CHANNELS:
            diver=x.pow(2).unsqueeze(1)
            diver=self.average(diver).squeeze(1)
            diver=diver.mul(self.alpha).add(1.0).pow(self.beta)
        else:
            diver=x.pow(2)
            diver=self.average(diver)
            diver=div.mul(self.alpha).add(1.0).pow(self.beta)
        x=x.div(diver)
        return x
            
            

In [34]:
NUM_CLASSES=10
class AlexNet(nn.Module):
    def __init__(self,num_classes=NUM_CLASSES):
        super(AlexNet,self).__init__()
        self.features=nn.Sequential(
            nn.Conv2d(3,64,kernel_size=4,stride=2,padding=1),#有问题
            nn.ReLU(inplace=True),
            #LRN(local_size=3, alpha=0.0001, beta=0.75),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64,192,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            #LRN(local_size=3, alpha=0.0001, beta=0.75),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192,384,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384,256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256,256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier=nn.Sequential(
            nn.Dropout(),
            nn.Linear(256*2*2,4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096,4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
            
    def forward(self,x):
        x=self.features(x)
        x=x.view(x.size(0),-1)#256*2*2
        x=self.classifier(x)
        return x
            
            
            
        
        
        
        
        

In [59]:
import torch.optim as optim
import torch.utils.data
from torch.utils.data import sampler
import torch.backends.cudnn as audnn
import torchvision
from torchvision import transforms
import numpy as np
import time
from tensorboardX import SummaryWriter

start=time.time()

writer=SummaryWriter()

lr=2.5e-4
epoch=100
trainBatchSize=100
testBatchSize=100
NUM_TRAIN = 49000
class Solver():
    def __init__(self):
        self.model=None
        self.lr=lr
        self.epochs=epoch
        self.train_batch_size=trainBatchSize
        self.test_batch_size=testBatchSize
        self.optimizer=None
        self.device=None
        self.cuda=torch.cuda.is_available()
        self.train_loader=None
        self.val_loader=None
        self.test_loader=None
        self.scheduler=None
        self.criterion =None
        
    def load_data(self):
        train_transform=transforms.Compose([transforms.RandomHorizontalFlip()，transforms.ToTensor()，transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])
        train_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=train_transform)
        self.train_loader=torch.utils.data.DataLoader(dataset=train_set,batch_size=self.train_batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
        val_set=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=train_transform)
        self.val_loader=torch.utils.data.DataLoader(dataset=train_set,batch_size=self.train_batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))
        test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform)
        self.test_loader=torch.utils.data.DataLoader(dataset=test_set,batch_size=self.test_batch_size,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))
        
    def load_model(self):
        if self.cuda:
            self.device=torch.device('cuda')#missed
        else:
            slf.device=torch.device('cpu')
            
        self.model=AlexNet().to(self.device)
        self.optimizer=optim.Adam(self.model.parameters(),lr=self.lr)
        self.scheduler=optim.lr_scheduler.MultiStepLR(self.optimizer,milestones=[75,150],gamma=0.5)
        self.criterion =nn.CrossEntropyLoss().to(self.device)
        
    def train(self):
        self.model.train()
        train_loss=0
        train_correct=0
        total=0
        
        for batch_num,(data,target) in enumerate(self.train_loader):
            data,target=data.to(self.device),target.to(self.device)
            self.optimizer.zero_grad()
            output=self.model(data)
            loss=self.criterion(output,target)
            loss.backward()
            self.optimizer.step()
            train_loss+=loss.item()
            prediction=torch.max(output,1)
            total+=target.size(0)
            train_correct += np.sum(prediction[1].cpu().numpy() == target.cpu().numpy())
            #print('loss:',loss.item(),'accuracy:',train_correct/total)
            
        return train_loss , train_correct/total
    
    def test(self):
        print('test====================')
        self.model.eval()
        test_loss=0
        test_correct=0
        total=0
        
        with torch.no_grad():
            for batch_num, (data, target) in enumerate(self.val_loader):
                data, target = data.to(self.device), target.to(self.device)
                output=self.model(data)
                loss=self.criterion(output,target)
                test_loss+=loss.item()
                prediction=torch.max(output,1)
                total+=target.size(0)
                test_correct += np.sum(prediction[1].cpu().numpy() == target.cpu().numpy())
                #print ('test accuracy:',test_correct/total)
        return test_loss,test_correct/total
    
    def run(self,writer):
        self.load_data()
        self.load_model()
        accuracy=0
        for epoch in range(1, self.epochs + 1):
            self.scheduler.step(epoch)
            print("\n=====> epoch: %d/200" % epoch)
            train_loss,train_result=self.train()
            print ('train accuracy:',train_result,'loss:',train_loss)
            test_result=self.test()
            accuracy=max(accuracy,test_result[1])
            print('test accuracy:',test_result[1])
            writer.add_scalar('loss_value',train_loss,epoch)
            if epoch == self.epochs:
                 print("===> BEST ACC. PERFORMANCE: %.3f%%" % (accuracy * 100))
        
        
        
    def final_test(self):
        self.model.eval()
        for x, y in  self.test_loader:
            x=x.to(self.device)
            y=y.to(self.device)
            scores=self.model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('FINNAL ACCURACY:',acc)
            
            

solver=Solver()
solver.run(writer)  
writer.close()
end=time.time()
print (end-start,"Final time")

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified

=====> epoch: 1/200
train accuracy: 0.3262857142857143 loss: 862.7013040781021
test====================
test accuracy: 0.443

=====> epoch: 2/200
train accuracy: 0.5115918367346939 loss: 647.1862459182739
test====================
test accuracy: 0.562

=====> epoch: 3/200
train accuracy: 0.5974081632653061 loss: 547.086560189724
test====================
test accuracy: 0.582

=====> epoch: 4/200
train accuracy: 0.6541632653061225 loss: 474.4081481695175
test====================
test accuracy: 0.628

=====> epoch: 5/200
train accuracy: 0.7043877551020408 loss: 409.82465732097626
test====================
test accuracy: 0.667

=====> epoch: 6/200
train accuracy: 0.7450204081632653 loss: 353.8947731554508
test====================
test accuracy: 0.686

=====> epoch: 7/200
train accuracy: 0.7884489795918367 loss: 296.36785411834717
test====================
test accuracy: 0.708

==

test accuracy: 0.718

=====> epoch: 65/200
train accuracy: 0.9928775510204082 loss: 11.166395108200959
test====================
test accuracy: 0.711

=====> epoch: 66/200
train accuracy: 0.9913061224489796 loss: 13.003872280125506
test====================
test accuracy: 0.726

=====> epoch: 67/200
train accuracy: 0.9917142857142857 loss: 12.453700906044105
test====================
test accuracy: 0.72

=====> epoch: 68/200
train accuracy: 0.9922244897959184 loss: 12.906551488966215
test====================
test accuracy: 0.722

=====> epoch: 69/200
train accuracy: 0.9929591836734694 loss: 11.114897075924091
test====================
test accuracy: 0.723

=====> epoch: 70/200
train accuracy: 0.9930816326530613 loss: 10.442134689910745
test====================
test accuracy: 0.723

=====> epoch: 71/200
train accuracy: 0.9918367346938776 loss: 12.389458281046245
test====================
test accuracy: 0.721

=====> epoch: 72/200
train accuracy: 0.9921224489795918 loss: 12.526631698783603
te

In [60]:


model=AlexNet()

    

solver.model.eval()
test_transform=transforms.Compose( [transforms.ToTensor()])
num_correct=0
num_samples=0
test_set=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=test_transform) 
test_loader=torch.utils.data.DataLoader(dataset=test_set,batch_size=100)
for x, y in  test_loader:
                x=x.to(solver.device)
                y=y.to(solver.device)
                scores=solver.model(x)
                _, preds = scores.max(1)
                num_correct += (preds == y).sum()
                num_samples += preds.size(0)
                acc = float(num_correct) / num_samples
x=x.to(torch.device("cpu"))
    
print('FINNAL ACCURACY:',acc)

Files already downloaded and verified
FINNAL ACCURACY: 0.7183


In [16]:
import torch.nn.functional as F
from tensorboardX import SummaryWriter
train_loss=0
train_correct=0
total=0
w=SummaryWriter(comment='alexnet')  
for batch_num,(data,target) in enumerate(solver.val_loader):
            data,target=data.to(solver.device),target.to(solver.device)
            solver.optimizer.zero_grad()
            output=solver.model(data)
            loss=solver.criterion(output,target)
            loss.backward()
            solver.optimizer.step()
            train_loss+=loss.item()
            prediction=torch.max(output,1)
            total+=target.size(0)
            train_correct += np.sum(prediction[1].cpu().numpy() == target.cpu().numpy())       
data = data.to(torch.device("cpu"))
model=AlexNet()
w.add_graph(model,(data,))
w.close()

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        self.bn = nn.BatchNorm2d(20)

    def forward(self, x):
        x = F.max_pool2d(self.conv1(x), 2)
        x = F.relu(x) + F.relu(-x)
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = self.bn(x)
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        return x


dummy_input = torch.rand(13, 1, 28, 28)

model = Net1()
with SummaryWriter(comment='Net1') as w:
    w.add_graph(model, (dummy_input,))